In [1]:
import pandas as pd
from keras import callbacks
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
inputs = pd.read_csv("../../BreastCancer/entradas_breast.csv")
outputs = pd.read_csv("../../BreastCancer/saidas_breast.csv")

In [3]:
# define callback
class Callback(callbacks.Callback):
    SHOW_NUMBER = 25
    
    epoch = 0

    # gives epoch feedback each 25 epochs
    def on_epoch_end(self, epoch, logs=None):
        self.epoch = epoch
        if (self.epoch % 25 == 0):
            print('Epoch: ' + str(self.epoch) + ' loss: ' + str(logs['loss']))

In [4]:
def createNetwork(
        layers_units,
        dropout,
        optimizer,
        loss,
        kernel_initializer,
        activation
    ):
    classifier = Sequential()
    classifier.add(Dense(
        units=layers_units[0],
        activation=activation,
        kernel_initializer=kernel_initializer,
        input_dim=30
    ))

    for units in layers_units[1:]:
    
        classifier.add(Dropout(dropout))
        
        classifier.add(Dense(
            units=units,
            activation=activation,
            kernel_initializer=kernel_initializer,
        ))

    classifier.add(Dropout(dropout))

    classifier.add(Dense(
        units=1,
        activation='sigmoid'  # critical parameter
    ))

    classifier.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['binary_accuracy']
    )

    return classifier

In [5]:
classifier = KerasClassifier(build_fn=createNetwork)
parameters = {
    'batch_size': [10, 30],
    'epochs': [50, 100],
    'layers_units': [[8, 8, 8], [32], [16, 8]],
    'dropout': [0.1, 0.2, 0.3],
    'optimizer': ['adam', 'ftrl', 'adamax'],
    'loss': ['binary_crossentropy'],
    'kernel_initializer': ['trucnated_normal', 'random_normal', 'normal'],
    'activation': ['relu', 'softmax', 'softplus']
}

C:\Users\Greze\AppData\Local\Temp\ipykernel_16024\2419956742.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn=createNetwork)


In [7]:
gridSearch = GridSearchCV(
    estimator=classifier,
    param_grid=parameters,
    scoring='accuracy',
    cv=10
)

gridSearch = gridSearch.fit(
    inputs,
    outputs,
    verbose=0,
    use_multiprocessing = True,
    callbacks=[Callback()]
)

Epoch: 0 loss: 0.7471906542778015


KeyboardInterrupt: 